# Citibike Project - Using DuckDB to analyze spatial relationships of the dataset

In [9]:
import duckdb
import leafmap
import polars as pl
import leafmap

## Import data

In [6]:
ride = pl.read_parquet('/Users/sra/files/projects/citibike_project/combined/group1_combined/group1_pl_ridenorm.parquet')
print('original shape: ', ride.shape)
ride.head()

original shape:  (55800085, 7)


ride_id,started_at,ended_at,start_station_id,end_station_id,ridertype,biketype
str,datetime[μs],datetime[μs],f64,f64,i32,i32
"""26A3DC47FE0EA3A3""",2021-05-13 12:48:08,2021-05-13 13:07:37,6173.08,5515.02,0,2
"""A99F2E1D627B088F""",2021-05-16 08:30:13,2021-05-16 08:45:47,6286.02,6873.01,0,2
"""43E79A45997B7390""",2021-05-01 08:38:14,2021-05-01 08:54:27,6286.02,6873.01,0,2
"""8B3CC649F4F58816""",2021-05-09 08:12:31,2021-05-09 08:27:05,6286.02,6873.01,0,2
"""7101C98F057486F4""",2021-05-27 07:52:27,2021-05-27 08:09:01,7636.05,7020.09,0,2


In [7]:
station = pl.read_parquet('/Users/sra/files/projects/citibike_project/combined/group1_combined/group1_pl_stationnorm.parquet')
print('original shape: ', station.shape)
station.head()

original shape:  (55800085, 4)


start_station_name,start_station_id,start_lat,start_lng
str,f64,f64,f64
"""Broadway & W 25 St""",6173.08,40.742868,-73.989186
"""46 Ave & 5 St""",6286.02,40.74731,-73.95451
"""46 Ave & 5 St""",6286.02,40.74731,-73.95451
"""46 Ave & 5 St""",6286.02,40.74731,-73.95451
"""E 123 St & Lexington Ave""",7636.05,40.802926,-73.9379


In [10]:
# group1_pl_intermediate=\
# group1_pl.with_columns(
#     pl.when(pl.col('member_casual') == 'member')
#     .then(pl.lit(0))
#     .otherwise(pl.lit(1))
#     .alias('ridertype'))

# group1_pl_intermediate.head()

# # named intermediate because this is an intermediate step
# # before finalizing this table of data

In [11]:
# group1_pl_intermediate=\
# group1_pl_intermediate.with_columns(
#         pl.when(pl.col('rideable_type') == 'classic_bike')
#         .then(pl.lit(0))
#         .when(pl.col('rideable_type') == 'electric_bike')
#         .then(pl.lit(1))
#         .otherwise(pl.lit(2))
#         .alias('biketype')
# )

# group1_pl_intermediate.head()

### Visualize the stations

In [ ]:

ride = ride.drop_nulls(subset=['county_fips'])
pres.head()

# filter for mode == "TOTAL"
pres = pres.filter(pl.col('mode') == 'TOTAL')

# pivot 'party' column to create separate columns
pres = pres.pivot(
    values=['candidatevotes', 'totalvotes'],  # columns to be spread across new columns
    index=['year', 'state', 'state_po', 'county_name', 'county_fips', 'office'],  # grouping columns
    columns='party',  # column whose values will become new column names
).rename({
    'candidatevotes_party_DEMOCRAT': 'd',
    'candidatevotes_party_REPUBLICAN': 'r',
    'candidatevotes_party_GREEN':'g',
    'candidatevotes_party_OTHER': 'o',
    'candidatevotes_party_LIBERTARIAN': 'l',
    'totalvotes_party_DEMOCRAT': 'total_d',
    'totalvotes_party_REPUBLICAN': 'total_r',
    'totalvotes_party_GREEN': 'total_g',
    'totalvotes_party_OTHER': 'total_o',
    'totalvotes_party_LIBERTARIAN': 'total_l'
}).drop(['office', 'state_po', 
         # 'county_name',  'state',
         'total_d', 'total_r', 'total_g', 'total_o', 'total_l',
         'g', 'o', 'l'
]).cast({'year': pl.Int32})

# pad the 'county_fips' column to ensure all strings are 5 digits
pres = pres.with_columns(
    pl.col("county_fips").cast(pl.Utf8).str.zfill(5).alias("county_fips")
)

# get order of colums
col_order = pres.columns

pres